In [64]:
import dolfinx
print(f"DOLFINx version: {dolfinx.__version__} based on GIT commit: {dolfinx.git_commit_hash} of https://github.com/FEniCS/dolfinx/")

DOLFINx version: 0.6.0 based on GIT commit:  of https://github.com/FEniCS/dolfinx/


In [80]:
from mpi4py import MPI
import numpy as np
from dolfinx import mesh, io
from dolfinx.fem import Function, FunctionSpace, dirichletbc, locate_dofs_topological, form
import ufl
from ufl import grad, div, nabla_grad, dx, inner, dot, Measure, SpatialCoordinate, TestFunctions, TrialFunctions, div, exp, inner

domain = mesh.create_unit_square(MPI.COMM_WORLD, 10, 10)

P2 = ufl.VectorElement("Lagrange", domain.ufl_cell(), 2)
P1 = ufl.FiniteElement("Lagrange", domain.ufl_cell(), 1)
TH = P2 * P1
V = FunctionSpace(domain, TH)
(v_a, p_a) = TrialFunctions(V)
(v_t, p_t) = TestFunctions(V)

x = SpatialCoordinate(domain)
U = 1.0



In [81]:
tdim = domain.topology.dim
fdim = tdim - 1
domain.topology.create_connectivity(fdim, tdim)
boundary_facets = mesh.exterior_facet_indices(domain.topology)


In [87]:
def u_exact(x):
    x_array = np.array(x[0])
    y_array = np.array(x[1])
    
    theta = np.arctan2(x_array, y_array)
    radius = (x[0]**2 + x[1]**2)**0.5
    Ue = -U
    theta0 = 0.5*np.pi
    f = Ue*(-theta0**2*np.sin(theta) + theta0*theta*np.sin(theta) \
           - np.sin(theta0)*np.cos(theta0)*theta*np.sin(theta) \
           + np.sin(theta0)**2*theta*np.cos(theta)) \
           /(theta0**2 - np.sin(theta0)**2)
    
    dfdt = Ue*(-theta0**2*np.cos(theta) + theta0*np.sin(theta) \
              + theta0*theta*np.cos(theta) - np.sin(theta0)*np.cos(theta0)*np.sin(theta) \
              - np.sin(theta0)*np.cos(theta0)*theta*np.cos(theta) \
              + np.sin(theta0)**2*np.cos(theta) - np.sin(theta0)**2*theta*np.sin(theta)) \
           /(theta0**2 - np.sin(theta0)**2)
    psie = radius*f
    vre = dfdt
    vte = -f
    ve = np.array([vre*np.cos(theta) - vte*np.sin(theta),
                   vre*np.sin(theta) + vte*np.cos(theta)])
    
    return ve


In [85]:
ve = u_exact(x)

AttributeError: 'Indexed' object has no attribute 'arctan2'

In [86]:
u_bc = Function(V)
u_bc.interpolate(ve)
bc_Bottom = dirichletbc(u_bc, dofs_BottomBoundary)
bc_Side = dirichletbc(u)
bc_TopRight = dirichletbc(u_bc, dofs_TopRightBoundary)
dofs_TopRightBoundary = locate_dofs_topological(V, TopRight, boundary_facets)
dofs_SideBoundary = locate_dofs_topological(V, Left, boundary_facets)
dofs_BottomBoundary = locate_dofs_topological(V, Bottom, boundary_facets)

n = FacetNormal(mesh)

K = inner(sym(grad(v_t)), sym(grad(v_a)))*dx 
G = -div(v_t)*p_a*dx
D = -p_t*div(v_a)*dx
S=K+G+D
L = Constant(domain, ScalarType(0.0)) * v_t * dx

#f = T_t*rhsf*dx + T_t*inner(gN, n)*ds
problem = LinearProblem(S, L, bcs=[bc_Bottom, bc_Side, bc_TopRight], petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
u_i = problem.solve()



NameError: name 've' is not defined

In [63]:
import numpy as np

theta = 0.5  # Example angle in radians
vre = 2  # Example vre value
vte = 3  # Example vte value

result = np.array([vre * np.cos(theta) - vte * np.sin(theta), vre * np.sin(theta) + vte * np.cos(theta)])
print(result)

[0.31688851 3.59159876]
